In [1]:
# NAME: DANG VIET QUAN
# TITLE: DECISION TREE
# CLASS: ICT-2018

In [2]:
import numpy as np
import pandas as pd

In [3]:
train =  pd.read_csv("train.csv")
test =  pd.read_csv("test.csv")
submission =  pd.read_csv('gender_submission.csv')

# Instruction:


## Attribute available in the dataset

In [4]:
print(train.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


### Catogorical attributes
Categorical: Survived, Name, Sex, Embarked. Ordinal: Pclass

### Numerical attributes
Continuous: Age, Fare. Discrete:SibSp, Parch

### Mixed attributes
Ticket is a mix of numeric and alphanumeric data types. Cabin is alphanumeric

In [5]:
# preview the data
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Attributes contain errors 
Name feature may contain errors as there are several ways used to describe a name including titles, round brackets and quotes used for alternative or short names

### Attributes data types:
- Seven attributes are numeric. Six in case of test data set
- Five atrributes are strings (object)

### Attributes contain missing values
- In trainning dataset: Age, Cabin, Embarked
- In test dataset: Age, Fair, Cabin

In [6]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

## 1 Correcting:

- Ticket attribute may be dropped from our analysis because there may not be a correlation between Ticket and survival.
- Cabin feature may be dropped as it is highly incomplete.
- PassengerId may be dropped from training dataset as it does not contribute to survival.
- Name may be dropped from our analysis because it is likely non-standard.


## 2 Completing
- We may want to complete Age feature as it is definitely correlated to survival by using mean value
- We may want to complete the Embarked feature by using mode value

## 3 Creating
- We may want to create a new feature called Num_mems based on Parch and SibSp to get total count of family members on board.


## 4 Transformation:
- We may transform Age and Fare into range

In [7]:
combine = [train, test]
for df in combine:
    df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1, inplace = True) # drop columns in correcting
    df['Age'].fillna(df['Age'].mean(), inplace= True) # completing data from (2)
    df['Fare'].fillna(df['Fare'].mean(), inplace= True) # completing data form (2)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
    df['num_mems'] = df['SibSp']+ df['Parch'] + 1 # create new column from (3)
    df.drop(['SibSp', 'Parch'], axis = 1, inplace = True) 
    df['Age']= pd.cut(df['Age'], bins = 10 , labels=np.arange(1,11),  right=False).astype('int') # split Age into 10 inteval and assign number
    df['Fare'] = pd.cut(df['Fare'], bins = 10, labels = np.arange(1,11), right = False).astype('int') # split Fare into 10 inteval and assign number
    df["Sex"] = pd.Categorical(df['Sex'], df['Sex'].unique()) # convert column Sex into catogory type
    df['Sex']  = df['Sex'].cat.rename_categories(np.arange(len(df['Sex'].unique()))) # convert into number
    df['Embarked'] = pd.Categorical(df['Embarked'], df['Embarked'].unique())
    df['Embarked'] = df['Embarked'].cat.rename_categories(np.arange(len(df['Embarked'].unique())))

In [8]:
class Question:
    """ Question includes name of column 
    """
    def __init__(self, column):
        self.column = column
class Leaf:
    """ Leaf includes label
    """
    def __init__(self, label):
        self.label = label
class Node:
    """ Node includes question, list of branches and label in current node
    """
    def __init__(self, question, list_branches, label):
        self.question = question
        self.list_branches = list_branches
        self.label = label

def partition(rows, question):
    """ 
    input: 
    rows is a dataframe and a question
    
    output:
    list of dataframes
    """
    L = {}
    unique = rows.iloc[:, question.column].unique().tolist()
    for value in unique:
        mask = rows.iloc[:, question.column] == value
        L[value] = rows[mask]
    return L

In [9]:
import math
def Entropy(rows):
    """
    Calculate entropy of current node
    """
    L = rows.iloc[:,0].value_counts()
    entro_value = 0
    S = rows.iloc[:,0].value_counts().sum()
    S =  float(S)
    for i in L.index:
        if L[i] == 0: 
            return 0
        entro_value -= L[i]/S * math.log(L[i]/S, 2)
    return entro_value      

In [10]:
def Gini(rows):
    """
    Calculate gini value of current node
    """
    L = rows.iloc[:,0].value_counts()
    gini_value = 0
    S = rows.iloc[:,0].value_counts().sum()
    S =  float(S)
    for i in L.index:
        if L[i] == 0: 
            return 0
        gini_value += (L[i]/S)**2
    return (1 - gini_value)

In [11]:
def Info_gain(rows, question, method):
    """
    Calculate information gain under individual question in current node
    """
    current = method(rows)
    L =  partition(rows, question)
    Tot = 0
    for child in L.values():
        Tot += len(child)/len(rows)*method(child)
    return current - Tot

In [12]:
def best_split(rows, method):
    """
    Find the question which can gain the most information gain in current node
    """
    numofcol = len(rows.iloc[0,:])
    best_gain = 0
    best_ques = None
    for col in range(1,numofcol):
        question = Question(col)
        currgain = Info_gain(rows, question, method)
        if currgain > best_gain:
            best_gain, best_ques = currgain, question
    return best_gain, best_ques

In [13]:
def build_tree(rows, depth, method):
    """
    input:
    rows: a dataframe
    depth: maximum of number hierarchy
    
    """
    best_gain, best_ques = best_split(rows, method)
    label = rows.iloc[:,0].value_counts().idxmax()
    if best_gain == 0:
        return Leaf(label)
    if depth == 0:
        return Leaf(label)
    children =  partition(rows, best_ques)
    List_branches = {}
    for i in list(children):
        List_branches[i] =  build_tree(children[i], depth - 1, method)
    return Node(best_ques, List_branches, label)

In [14]:
def classify(node,row):
    """
    input:
    node: 
    row: row of data
    output:
    classifier of row
    """
    if isinstance(node, Leaf):
        return node.label
    val = row[node.question.column]
    if val in list(node.list_branches):
        newnode = node.list_branches[val]
    else:
        return node.label
    return classify(newnode, row)

In [15]:
def main_tree(X,y, depth_max, method):
    X_train =  X.copy()
    X_train.insert(loc=0, column='label', value = y)
    tree = build_tree(X_train, depth_max, method)
    return tree

In [16]:
def fit(tree, x):
    x_test = x.copy()
    x_test.insert(loc = 0, column  = 'label', value = 0)
    predicts = pd.Series(np.zeros(len(x_test)).astype('int'))
    for i in range(len(x_test)):
        predicts[i]=  classify(tree, x_test.iloc[i,:])
    return predicts

In [17]:
from graphviz import Digraph
global list_att
list_att = train.columns.values
def print_branch(graph, node, node_name = 'T'):
    """
    Visualize the current node
    """
    for i in list(node.list_branches):
        if isinstance(node.list_branches[i], Leaf):
            current_name = node_name + 'L' + str(i)
            graph.node(current_name,str(node.list_branches[i].label), shape = 'diamond')
            graph.edge(node_name, current_name, label = str(i))
#             print(current_name, node_name)
        else:
            current_name =  node_name + str(i)
            graph.node(current_name, list_att[node.list_branches[i].question.column], shape = 'circle')
            graph.edge(node_name, current_name, label = str(i))
#             print(current_name, node_name)
            print_branch(graph, node.list_branches[i], current_name)
def print_tree(tree):
    """
    Visualize the tree
    """
    graph =  Digraph(comment = 'Decision Tree')
    graph.attr(rankdir='LR', size='1.0')
    graph.node('root', list_att[tree.question.column])
    print_branch(graph, tree, 'root')
    return graph

In [20]:
# def prune(node):
#     """
#     prune the tree
#     """
#     current_label = node.label
#     switch = 0
#     for i in list(node.list_branches):
#         if node.list_branches[i].label != current_label or isinstance(node.list_branches[i], Leaf) is False:
#             switch = 1
#             break
#     if switch == 0:
#         node.list_branches = {}
#         return 
#     for i in list(node.list_branches):
#             if isinstance(node.list_branches[i], Leaf):
#                 continue
#             else:    
#                 prune(node.list_branches[i])
# def refresh_tree(node, tree):
#     """
#     refresh the tree after prune
#     """
#     if isinstance(node, Leaf):
#         return
#     for i in list(node.list_branches):
#         if isinstance(node.list_branches[i], Leaf):
#             continue
#         if len(node.list_branches[i].list_branches) == 0:
#             node.list_branches[i] = Leaf(node.list_branches[i].label)
#             prune(tree)
#         else:
#             refresh_tree(node.list_branches[i], tree)
def prune(node):
    current_label  = node.label
    switch = 0
    for i in list(node.list_branches):
        if node.list_branches[i].label != current_label or isinstance(node.list_branches[i], Leaf) is False:
            switch = 1
            break
    if switch == 0:
        node.list_branches = {}
        return
    for i in list(node.list_branches):
        if isinstance(node.list_branches[i], Leaf) is False:
            if len(node.list_branches[i].list_branches) == 0:
                node.list_branches[i] = Leaf(node.list_branches[i].label)
    for i in list(node.list_branches):
            if isinstance(node.list_branches[i], Leaf):
                continue
            else:    
                prune(node.list_branches[i])

In [22]:
x_train = train.copy()
y_train = x_train.pop('Survived')
# create the tree with train dataset
tree = main_tree(x_train, y_train, depth_max = 6, method = Entropy)
# prune tree
depth_max = 6
for i in range(depth_max):
    prune(tree)
a= print_tree(tree)
a.view()

'Digraph.gv.pdf'

In [ ]:
# predictions
predicts = fit(tree, test)
predicts.value_counts()

In [ ]:
# calculate accuracy
(predicts == submission.Survived).sum()/len(predicts)